In [1]:
using Plots
using DifferentialEquations
using Interact
using WebIO

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1587242135283340870\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3835275883120391697\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1587242135283340870\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
sir_ode = @ode_def SIRModel begin
    dS = -β*S*I
    dI = β*S*I-γ*I
    dR = γ*I
    end β γ

(::SIRModel{var"###ParameterizedDiffEqFunction#428",var"###ParameterizedTGradFunction#429",var"###ParameterizedJacobianFunction#430",Nothing,Nothing,ModelingToolkit.ODESystem}) (generic function with 1 method)

In [3]:
@manipulate throttle=0.01 for 
    conf=slider(30:1:200,label="Duration of Quarantine"), 
    c3=slider(5:0.1:15,label="Contact Coefficient"),
    p3=slider(0.005:0.001:0.03,label="Probability of Infection"), 
    Tglob=slider(100:1:500, label="Length")
######PARAMETERS#####

#Global parameters
n=67000000.0 #Population
I0=100.0 #Infected at start
d=15.0 #Infectious period
ν=1/d #Transmission factor
##Tglob=200.0 #Number of days modelled
#conf=45
div=15.0 #Day when measures are implemented
#hmax=0.006 #Maximal Capacity of hospitals

#Initial Scenario
c1=13.4 #Contact coefficient
p1=0.02 #Probability of infection in contact
τ1=c1*p1 #β parameter
#########
Tmax=round(Int64,div) 
tend=convert(Int64, Tmax*10+1)
t=1:tend
parms = [τ1,ν] 
init = [1-I0/n,I0/n,0.0]
tspan = (0.0,Tmax)
########

#Scenario after shock
c2=c1*0.9 #contact coefficient
p2=p1*0.8 #probability of infection in contact
τ2=c2*p2 #β parameter
#########
Tmax2=round(Int64,conf)
parms2 = [τ2,ν] 
tspan2 = (0.0,Tmax2)
tend2=convert(Int64, Tmax2*10+1)
t2=tend+1:tend2+tend
#########
    
#End of Quarantine Scenario
#c3=c1 #contact coefficient
#p3=p1 #probability of infection in contact
τ3=c3*p3 #β parameter
#########
Tmax3=round(Int64,Tglob-conf-Tmax)
parms3 = [τ3,ν] 
tspan3 = (0.0,Tmax3)
tend3=convert(Int64, Tmax3*10+1)
t3=tend2+1+tend:tend3+tend2+tend
#########

#Counterfactual
tend4=convert(Int64, Tglob*10+1)
t4=1:tend4
tspan4 = (0.0,Tglob)
##################################

#####Solving######

#Initial ODE system
sir_prob = ODEProblem(sir_ode,init,tspan,parms)
sir_sol = DifferentialEquations.solve(sir_prob,saveat = 0.1)
#Storing results
Inew=fill(0.0,tend)
Snew=fill(0.0,tend)
Rnew=fill(0.0,tend)
for i in 1:tend
    Snew[i]=sir_sol.u[i][1]
    Inew[i]=sir_sol.u[i][2]
    Rnew[i]=sir_sol.u[i][3]
end
#####

#New Initial values
I02=Inew[tend]
R02=Rnew[tend]
S02=Snew[tend]
init2 = [S02,I02,R02]
#After Shock ODE system
sir_prob2 = ODEProblem(sir_ode,init2,tspan2,parms2)
sir_sol2 = DifferentialEquations.solve(sir_prob2,saveat = 0.1)
#Storing results
Inew2=fill(0.0,tend2)
Snew2=fill(0.0,tend2)
Rnew2=fill(0.0,tend2)
for i in 1:tend2
    Snew2[i]=sir_sol2.u[i][1]
    Inew2[i]=sir_sol2.u[i][2]
    Rnew2[i]=sir_sol2.u[i][3]
end
I03=Inew2[tend2]
R03=Rnew2[tend2]
S03=Snew2[tend2]
init3 = [S03,I03,R03]
sir_prob3=ODEProblem(sir_ode,init3,tspan3,parms3)
sir_sol3=DifferentialEquations.solve(sir_prob3,saveat=0.1)
Inew3=fill(0.0,tend3)
Snew3=fill(0.0,tend3)
Rnew3=fill(0.0,tend3)
for i in 1:tend3
    Snew3[i]=sir_sol3.u[i][1]
    Inew3[i]=sir_sol3.u[i][2]
    Rnew3[i]=sir_sol3.u[i][3]
end
    
St=vcat(Snew,Snew2,Snew3)
It=vcat(Inew,Inew2,Inew3)
Rt=vcat(Rnew,Rnew2,Rnew3)
tt=vcat(t./10,t2./10,t3./10)
#####

#Counterfactual ODE System
sir_prob4 = ODEProblem(sir_ode,init,tspan4,parms)
sir_sol4 = DifferentialEquations.solve(sir_prob4,saveat = 0.1)
#Storing results
Inew4=fill(0.0,tend4)
Snew4=fill(0.0,tend4)
Rnew4=fill(0.0,tend4)
for i in 1:tend4
    Snew4[i]=sir_sol4.u[i][1]
    Inew4[i]=sir_sol4.u[i][2]
    Rnew4[i]=sir_sol4.u[i][3]
end
t4=t4./10
##############

######Plots######

#Quarantine scenario
plot(tt,St*100,label="Susceptible",color=:blue,width=3,grid=false) #Susceptible
plot!(tt,It*100,label="Infected",color=:red,width=3) #Infected
plot!(tt,Rt*100,label="Removed",color=:green,width=3) #Removed
#Counterfactual scenario
plot!(t4,Snew4*100,label="",color=:blue,linestyle=:dash) #Susceptible
plot!(t4,Inew4*100,label="",color=:red,linestyle=:dash) #Infected
plot!(t4,Rnew4*100,label="",color=:green,linestyle=:dash) #Removed
#Reference lines
plot!([findall(x->x==maximum(It),It)[1][1]/10],seriestype="vline",label="Peak of Infections",color=:red, linestyle=:solid, width=2)
plot!([Tmax],seriestype="vline",label="Start of Quarantine",color=:black,linealpha=0.5,width=2) #Implementation date
plot!([conf+Tmax],seriestype="vline",label="End of Quarantine",color=:black,linestyle=:dot,width=2)#End of quarantine date
#plot!([hmax],seriestype="hline",label="",color=:black,linestyle=:dot,width=2) #Hospital Capacity
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Duration of Quarantine"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 171,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
86, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/danell/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/danell/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/danell/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/danell/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/danell/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000013b85cb50, Task (runnable) @0x000000013b85cb50), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40\",\"41\",\"42\",\"43\",\"44\",\"45\",\"46\",\"47\",\"48\",\"49\",\"50\",\"51\",\"52\",\"53\",\"54\",\"55\",\"56\",\"57\",\"58\",\"59\",\"60\",\"61\"